## Imports

In [3]:
import pyxdf 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from lmfit.models import Model
from scipy import stats
import pywt
import math

## Function Definitions

Predicted pupil dilation, $d(Y)$, caused by luminance $Y$, is computed with the following equation: $𝑑(𝑌) = 𝑎 · 𝑒^{−𝑏·𝑌} + c$

In [4]:
def pupil_func(x, a, b, c):
    return a * np.exp(-b * x) + c

In [5]:
def import_data(file):
    streams, header = pyxdf.load_xdf(file)
    dfs = {}
    for stream in streams:
        stream_name = stream['info']['name'][0]
        stream_channels = {channel['label'][0]: i for i, channel in enumerate(stream['info']['desc'][0]['channels'][0]['channel'])}
        stream_data = stream['time_series']
        data_dict = {key: np.array(stream_data)[:, index] for key, index in stream_channels.items()}
        data_dict['time'] = np.round(np.array(stream['time_stamps']), decimals=4)
        dfs[stream_name] = pd.DataFrame(data_dict).drop_duplicates(subset=['time']).reset_index(drop=True)
    return dfs

In [6]:
accom_time = pd.to_timedelta(0.5, unit='s')

In [7]:
def process_gaze_luminance_data(stream_df):
    pupil = stream_df['GazeStream'].loc[(stream_df['GazeStream']['LeftEyeIsBlinking'] == 0) & (stream_df['GazeStream']['RightEyeIsBlinking'] == 0) & (stream_df['GazeStream']['LeftPupilDiameter'] > 0) & (stream_df['GazeStream']['RightPupilDiameter'] > 0), ['time', 'MethodID', 'ModelID', 'LeftPupilDiameter', 'RightPupilDiameter']]
    pupil['time'] = pd.to_timedelta(pupil['time'], unit='s')

    lum = stream_df['LuminanceStream'].loc[:, ['time', 'MethodID', 'ModelID', 'Luminance']]
    lum['time'] = pd.to_timedelta(lum['time'], unit='s')

    # Intersection of time stamps
    pupil_lum_time_intersection = np.intersect1d(pupil['time'], lum['time'])

    # Filter pupil and luminance data by intersection
    pupil = pupil[pupil['time'].isin(pupil_lum_time_intersection)].reset_index(drop=True)
    lum = lum[lum['time'].isin(pupil_lum_time_intersection)].reset_index(drop=True)

    # Combined DataFrame for pupil and luminance
    pupil_lum = pd.DataFrame({
        'time': pd.to_timedelta(lum['time'], unit='s'),
        'luminance': lum['Luminance'],
        'pupilDiameter': 0.5 * (pupil['LeftPupilDiameter'] + pupil['RightPupilDiameter']),
        'methodID': pupil['MethodID'],
        'modelID': pupil['ModelID']
    }).resample('0.01s', on='time').mean()

    pupil_lum['time'] = pupil_lum.index

    return pupil_lum

In [8]:
def process_calibration_data(pupil_lum_df, stream_df):
    calibration_events = stream_df['ExperimentStream'].loc[(stream_df['ExperimentStream']['EventType'] == 'CalibrationColorChange') | (stream_df['ExperimentStream']['SceneEvent'] == 'Calibration') | (stream_df['ExperimentStream']['SceneEvent'] == 'CalibrationComplete'), ['time','SceneEvent', 'EventType']]
    calibration_events['time'] = pd.to_timedelta(calibration_events['time'], unit='s')
    c_start_times = calibration_events[:8]['time']
    c_end_times = calibration_events[1:]['time']
    c_start_times.reset_index(drop=True, inplace=True)
    c_end_times.reset_index(drop=True, inplace=True)

    calib_data = {}
    for i in range(8):
        calib_data[i] = pupil_lum_df.loc[(pupil_lum_df['time'] >= c_start_times[i]) & (pupil_lum_df['time'] <= c_end_times[i]), ['time','luminance', 'pupilDiameter']]
        calib_data[i]['time'] -= calib_data[i]['time'].iloc[0]
        calib_data[i] = calib_data[i].loc[(calib_data[i]['time'] >= accom_time), ['luminance', 'pupilDiameter']]

    calibration_data = pd.concat(calib_data).groupby(level=0).mean().sort_values(by=['luminance']).reset_index(drop=True)
    return calibration_data

In [9]:
def process_navigation_data(pupil_lum_df, stream_df, a, b, c):
    navigation_events = stream_df['ExperimentStream'].loc[(stream_df['ExperimentStream']['SceneEvent'] == 'NavigationComplete') | (stream_df['ExperimentStream']['SceneEvent'] == 'Navigation_Trial'), ['time','SceneEvent', 'EventType', 'ModelID', 'MethodID']]
    navigation_events['time'] = pd.to_timedelta(navigation_events['time'], unit='s')
    nav_start_times = navigation_events.loc[navigation_events['SceneEvent'] == 'Navigation_Trial', 'time']
    nav_end_times = navigation_events.loc[navigation_events['SceneEvent'] == 'NavigationComplete', 'time']

    nav_start_times.reset_index(drop=True, inplace=True)
    nav_end_times.reset_index(drop=True, inplace=True)

    #Correct for occasions when Unity emitted multiple SceneLoaded events for a single trial
    if len(nav_start_times) > 8:
        nav_diff = nav_start_times.diff().dt.total_seconds()
        nav_start_times = nav_start_times.loc[(nav_diff.isnull()) | (nav_diff > 3)]

    nav_start_times.reset_index(drop=True, inplace=True)
    nav_end_times.reset_index(drop=True, inplace=True)

    nav_data = {}
    for i in range(8):
        nav_data[i] = pupil_lum_df.loc[(pupil_lum_df['time']>=nav_start_times.loc[i]) & (pupil_lum_df['time']<=nav_end_times.loc[i]), ['time', 'methodID', 'modelID', 'luminance', 'pupilDiameter']]
        nav_data[i].set_index('time', inplace=True, drop=False)

    navigation_data = pd.concat(nav_data, names=['trial'])
    navigation_data = navigation_data.groupby(level=0).resample('0.01s', on='time', ).mean()
    navigation_data['plr'] = pupil_func(navigation_data['luminance'], a, b, c)
    navigation_data['tepr'] = navigation_data['pupilDiameter'] - navigation_data['plr']
    
    return navigation_data

In [10]:
def modmax(d):
    # compute signal modulus
    m = [0.0]*len(d)
    for i in range(len(d)):
        m[i] = math.fabs(d[i])
    # if value is larger than both neighbours , and strictly larger than either , then it is a local maximum
    t = [0.0]*len(d)
    for i in range(len(d)):
        ll = m[i -1] if i >= 1 else m[i]
        oo = m[i]
        rr = m[i+1] if i < len(d)-2 else m[i]
        if (ll <= oo and oo >= rr) and (ll < oo or oo > rr):
        # compute magnitude
            t[i] = math.sqrt(d[i]**2)
        else:
            t[i] = 0.0
    return t

In [11]:
def ipa(d):
    # obtain 2-level DWT of pupil diameter signal d
    try:
        (cA2 ,cD2 ,cD1) = pywt.wavedec(d,'sym16', 'per', level=2)
    except ValueError :
        return
    # get signal duration (in seconds)
    tt = d.index[-1] - d.index[0]
    # normalize by 1/2 j , j = 2 for 2-level DWT
    cA2 [:] = [x / math.sqrt (4.0) for x in cA2]
    cD1 [:] = [x / math.sqrt (2.0) for x in cD1]
    cD2 [:] = [x / math.sqrt (4.0) for x in cD2]

    # detect modulus maxima , see Listing 2
    cD2m = modmax(cD2)
    # threshold using universal threshold λuniv = σˆp(2logn)
    # where σˆ is the standard deviation of the noise
    λuniv = np.std(cD2m) * math.sqrt (2.0* np.log2(len(cD2m )))
    cD2t = pywt. threshold (cD2m ,λuniv,mode="hard")
    # compute IPA
    ctr = 0
    for i in range(len(cD2t )):
        if math.fabs(cD2t[i]) > 0: ctr += 1
    IPA = float(ctr)/tt.total_seconds()

    return IPA


## Load Data

Load the data from the xdf file for a single participant.

In [12]:
file = './Path_Data/ID_114.xdf'
df = import_data(file)

In [13]:
pupil = df['GazeStream'].loc[(df['GazeStream']['LeftEyeIsBlinking'] == 0) & (df['GazeStream']['RightEyeIsBlinking'] == 0) & (df['GazeStream']['LeftPupilDiameter'] > 0) & (df['GazeStream']['RightPupilDiameter'] > 0), ['time', 'MethodID', 'ModelID', 'LeftPupilDiameter', 'RightPupilDiameter']]
pupil['time'] = pd.to_timedelta(pupil['time'], unit='s')

lum = df['LuminanceStream'].loc[:, ['time', 'MethodID', 'ModelID', 'Luminance']]
lum['time'] = pd.to_timedelta(lum['time'], unit='s')

# Intersection of time stamps
pupil_lum_time_intersection = np.intersect1d(pupil['time'], lum['time'])

# Filter pupil and luminance data by intersection
pupil = pupil[pupil['time'].isin(pupil_lum_time_intersection)].reset_index(drop=True)
lum = lum[lum['time'].isin(pupil_lum_time_intersection)].reset_index(drop=True)

# Combined DataFrame for pupil and luminance
pupil_lum_df = pd.DataFrame({
    'time': pd.to_timedelta(pupil_lum_time_intersection, unit='s'),
    'luminance': lum['Luminance'],
    'pupilDiameter': 0.5 * (pupil['LeftPupilDiameter'] + pupil['RightPupilDiameter']),
    'methodID': pupil['MethodID'],
    'modelID': pupil['ModelID']
})


In [14]:
calibration_events = df['ExperimentStream'].loc[(df['ExperimentStream']['EventType'] == 'CalibrationColorChange') | (df['ExperimentStream']['SceneEvent'] == 'Calibration') | (df['ExperimentStream']['SceneEvent'] == 'CalibrationComplete'), ['time','SceneEvent', 'EventType']]
calibration_events['time'] = pd.to_timedelta(calibration_events['time'], unit='s')
c_start_times = calibration_events[:8]['time']
c_end_times = calibration_events[1:]['time']
c_start_times.reset_index(drop=True, inplace=True)
c_end_times.reset_index(drop=True, inplace=True)

calib_data = {}
for i in range(8):
    calib_data[i] = pupil_lum_df.loc[(pupil_lum_df['time'] >= c_start_times[i]) & (pupil_lum_df['time'] <= c_end_times[i]), ['time','luminance', 'pupilDiameter']]
    calib_data[i]['time'] -= calib_data[i]['time'].iloc[0]
    calib_data[i] = calib_data[i].loc[(calib_data[i]['time'] >= accom_time), ['luminance', 'pupilDiameter']]

calibration_data = pd.concat(calib_data).groupby(level=0).mean().sort_values(by=['luminance']).reset_index(drop=True)


In [15]:
x_data = calibration_data['luminance']
y_data = calibration_data['pupilDiameter']
exp_mod = Model(pupil_func)
params = exp_mod.make_params(a=1, b=4, c=0)
result = exp_mod.fit(y_data, params, x=x_data)
a = result.params['a'].value
b = result.params['b'].value
c = result.params['c'].value

In [126]:
nav_start_times = pd.to_timedelta(df['NavigationStream'].groupby(['ModelID', 'MethodID']).first()['time'], unit='s')
nav_start_times.reset_index(drop=True, inplace=True)

df['SurveyStream'] = df['SurveyStream'].replace(r'^\s*$', np.nan, regex=True).dropna()
df['SurveyStream']['ModelID'] = df['SurveyStream']['ModelID'].astype(float)

nav_end_times = df['SurveyStream'].loc[(df['SurveyStream']['SurveyType'] == 'Discomfort') & (df['SurveyStream']['ModelID'] < 4), ['time', 'ModelID', 'MethodID']]
nav_end_times.reset_index(drop=True, inplace=True)
nav_end_times['time'] = pd.to_timedelta(nav_end_times['time'], unit='s') - pd.offsets.Second(3)

nav_end_times.reset_index(drop=True, inplace=True)
print(nav_start_times)
print(nav_end_times)


nav_data = {}
for i in range(8):
    nav_data[i] = pupil_lum_df.loc[(pupil_lum_df['luminance'] > 0) 
                                   & (pupil_lum_df['modelID']< 4)
                                   & (pupil_lum_df['time']>nav_start_times.loc[i]) 
                                   & (pupil_lum_df['time']<nav_end_times.loc[i]), ['time', 'methodID', 'modelID', 'luminance', 'pupilDiameter']]
    nav_data[i].reset_index('time', inplace=True, drop=False)


navigation_data = pd.concat(nav_data, names=['trial'])
navigation_data['plr'] = pupil_func(navigation_data['luminance'], a, b, c)
navigation_data['adj_pupil_d'] = navigation_data['pupilDiameter'] - navigation_data['plr']

0   0 days 23:40:32.816500
1   0 days 23:46:29.833200
2   0 days 23:41:43.825900
3   0 days 23:47:50.628800
4   0 days 23:42:56.892300
5   0 days 23:48:47.933700
6   0 days 23:44:07.401900
7   0 days 23:50:06.562500
Name: time, dtype: timedelta64[ns]
                    time  ModelID MethodID
0 0 days 23:41:28.288000      0.0        2
1 0 days 23:42:44.773800      1.0        2
2 0 days 23:43:54.695300      2.0        2
3 0 days 23:45:09.681900      3.0        2
4 0 days 23:47:38.365200      0.0        3
5 0 days 23:48:35.671900      1.0        3
6 0 days 23:49:53.845200      2.0        3
7 0 days 23:50:48.341400      3.0        3


ValueError: Can only compare identically-labeled Series objects

In [92]:
methodIds = []
ipaVals = []

d = navigation_data.groupby('trial')
for name, group in d:
    print(name)

    



0
2
4
6
7
